In [ ]:
"""
Student Information Management System
This module provides functionality to manage student data including reading from a file,
displaying information, and updating favorite programs through an interactive interface.
"""

from google.colab import drive
import ipywidgets as gui_widgets
from IPython.display import display

# Mount Google Drive for persistent storage
drive.mount('/content/drive')

# Define path to student data file
data_file_location = "/content/student.txt"

# Load student data from file
try:
    with open(data_file_location, "r", encoding="utf-8") as input_file:
        raw_data = input_file.read()
except FileNotFoundError:
    print("⚠️ Error: Data file not found.")

# Process and store student information in dictionary
student_database = {}

for data_line in raw_data.strip().split("\n"):
    data_segments = data_line.strip().split(",")
    if len(data_segments) >= 2:
        student_name = data_segments[0].strip()
        student_info = ", ".join(data_segments[1:]).strip()
        student_database[student_name] = student_info

# Create GUI components
student_selector = gui_widgets.Dropdown(
    options=["Select Student"] + list(student_database.keys()),
    description="Student:",
)

info_display = gui_widgets.Output()

program_input = gui_widgets.Text(
    description="Favorite Program:",
    placeholder="Enter favorite program",
)

save_button = gui_widgets.Button(description="Update Program")

def update_student_program(student_id, program_name):
    """
    Updates a student's favorite program in the database and file.
    
    Args:
        student_id (str): The student's name/identifier
        program_name (str): The new favorite program to be added
    """
    if student_id in student_database:
        current_info = student_database[student_id]
        info_fields = current_info.split(", ")
        info_fields = [field for field in info_fields if "תוכנית אהובה" not in field]
        info_fields.append(f"תוכנית אהובה: {program_name}")
        student_database[student_id] = ", ".join(info_fields)

        # Save updated information to file
        with open(data_file_location, "w", encoding="utf-8") as output_file:
            for sid, info in student_database.items():
                output_file.write(f"{sid}, {info}\n")

        print(f"✅ Program {program_name} successfully updated for student {student_id}.")
    else:
        print(f"⚠️ Error: Student {student_id} not found in database.")

def handle_selection(change):
    """
    Handles the event when a student is selected from the dropdown.
    Updates the display with student information and shows update controls.
    
    Args:
        change (dict): The change event data containing the selected value
    """
    with info_display:
        info_display.clear_output()
        if change["new"] in student_database:
            print(f"Student Details:\n{student_database[change['new']]}")
            display(program_input)
            program_input.value = ""
            display(save_button)

def handle_update(button_click):
    """
    Handles the update button click event.
    Updates the student's favorite program if valid input is provided.
    
    Args:
        button_click: The button click event object
    """
    selected_id = student_selector.value
    new_program = program_input.value
    if selected_id != "Select Student" and new_program:
        update_student_program(selected_id, new_program)

# Connect event handlers
save_button.on_click(handle_update)
student_selector.observe(handle_selection, names="value")

# Display the interface
display(student_selector, info_display)
